## Data Scraping with Python's BeautifulSoup

Obtaining data is a very time consuming and difficult task but with web scraping we can make the task much faster and easier. 
This is a guide on how I scraped data from the OUA (Ontario University Athletics) website with Python and the Python library, BeautifulSoup. Doing this task requires some Python, and HTML knowledge.

We'll look at the website to get a feel of what data we want and where it is coming from.
I was going through this site http://www.oua.ca/sports/mbkb/2018-19/leaders for the men's basketball teams and in the top right you can see a drop down menu where you can select the Season you want to view data from. This is how the webpage looks:

<img src="images/webpage.png"/>

After surfing the webpage for desired data, I came across a goldmine.
In this table we can select a team and view many different statistics that were collected.

<img src="images/table.png"/>

If we click on a team we can find a game log for the entire regular season when we click on the 'Game Log' tab.

<img src="images/gamelog.png"/>

From here we can see that each score has an embedded hyperlink and if we click it we can get the game stats. There are player stats of the game, play by play data, team stats, and 1st and 2nd half stats. What I was interested in was the team stats (I'll soon also scrape the player stats and possibly play-by-play data).

<img src="images/teamstats.png"/>

This is the goldmine right here. We have the game stats for each team and multiple useful tables that we can use. 
Now this is where the HTML comes in. What we want to do now is inspect elements on the page so that we can see the source code.

<img src="images/htmltable.png"/>

This is where we can see the HTML and its structure. In this page source we can see there are different 'sections'; these are the different tabs (e.g. Box Score tab, Play by Play tab, etc) and in each one there is a table. To scrape the data we need to figure out where the data comes from so that we can reference it and tell Python that we want it.

In this table we have a lot of variables like Field Goal for Away and Home team, 3 Point, Turnovers, etc. So we want to find the tags that reference each of them and use some indexing.

<img src="images/indexing.png"/>

Now that we know that the data is shown in the html, we can use Python to find the html tags and collect them.

First we need to start with importing the appropriate library packages.

In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from collections import defaultdictr = requests.get(url[i])

In [5]:
r = requests.get('http://www.oua.ca/sports/mbkb/2018-19/boxscores/20180929_svqd.xml?view=teamstats')
#this lets us access the webpage that we want to scrape from
raw_html = r.content
#we can then get the html
soup = BeautifulSoup(raw_html, 'html.parser')
#soup is what we call the html of the webpage which we can use to scrape
stats = soup.findAll("table")
print(stats[8])

<table>
<caption class="caption offscreen"><h2>Team Statistics</h2></caption>
<tr>
<th class="col-head" scope="col"><span class="offscreen">Stat</span></th>
<th class="col-head" scope="col">Algoma</th>
<th class="col-head" scope="col">Laurentian</th>
</tr>
<tr>
<th class="row-head text" scope="row">Field Goal</th>
<td>            31-64
    </td>
<td>            26-51
    </td>
</tr>
<tr>
<th class="row-head text" scope="row">Field Goal %</th>
<td>            48.4%
    </td>
<td>            51.0%
    </td>
</tr>
<tr>
<th class="row-head text" scope="row">3 Point</th>
<td>            11-23
    </td>
<td>            7-18
    </td>
</tr>
<tr>
<th class="row-head text" scope="row">3 Point %</th>
<td>            47.8%
    </td>
<td>            38.9%
    </td>
</tr>
<tr>
<th class="row-head text" scope="row">Free Throw</th>
<td>            8-9
    </td>
<td>            10-13
    </td>
</tr>
<tr>
<th class="row-head text" scope="row">Free Throw %</th>
<td>            88.9%
    </td>
<td>      

This is the HTML of the table we want to scrape and we can index different tags to get the desired data.

In [7]:
table = stats[8]
print(table.findAll('th', {'scope': 'col'})[1].text.strip()) #This will find the second "th" tag with the scope equal to "col" and this prints the Away team 
print(table.findAll('th', {'scope': 'col'})[2].text.strip()) #This will print the Home team

Algoma
Laurentian


This table is good but it doesn't provide the total scores from the game. However we do have the scores at the top of the page.

<img src="images/score.png"/>

We can use BeautifulSoup to obtain these values as well. After figuring out where all the values are on the webpage, we can create a Python dictionary with keys as our variable names and values as our data that we obtain from the table. 

Below is a general function that scrapes the data of an OUA game webpage into a dataframe.

In [ ]:
def scrape(url):
    """ This function is used to create data dictionaries for any url of team stats in the oua website. It
    takes an array of urls but for some games there are extra fields to look out for. This function is for those that
    do not have those extra fields in the table"""

    # create dictionary for links with less fields in table
    dictlist = {} # initializes a dictionary
    for i in range(len(url)):
    # loop through all the urls in an array
        print(url[i])
        r = requests.get(url[i])
        raw_html = r.content
        soup = BeautifulSoup(raw_html, 'html.parser')
        soup[url[i]] = BeautifulSoup(raw_html, 'html.parser')
        stats = soup[url[i]].findAll("table")
        scores = soup[url[i]].findAll('div', {'class': 'teams clearfix'})[0].table
        # some links have different amounts of tables and sometimes the team stats table is different
        table = stats[8]
        for j in range(2, len(stats)):
            if str(stats[j].caption) == '<caption class="caption offscreen"><h2>Team Statistics</h2></caption>':
                table=stats[j]
                break
        # this makes sure we get the table we want since it will get the html with the certain caption class (a unique part of the html)

        dictlist[url[i]] = {}
        d = {}
        # we will then create the dictionary with the proper keys and values
        dictlist[url[i]] = {
            "Away" : table.findAll('th', {'scope': 'col'})[1].text.strip(),
            "Home" : table.findAll('th', {'scope': 'col'})[2].text.strip(),

        }
        # some webpages may not have the values listed below so we'll have to watch out for that and prevent getting an index error
        try:
            winner = scores.findAll('tr', {'class': 'winner'})[0]
        except IndexError:
            d[None] = None
        # if the tag is not found we create a null key and value so that it doesn't break
        try:
            loser = scores.findAll('tr', {'class': 'loser'})[0]
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update({"Winner": winner.th.text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update({"Loser": loser.th.text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update({"Winner 1st Qtr Pts": winner.findAll('td')[0].text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update({"Loser 1st Qtr Pts": loser.findAll('td')[0].text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update({"Winner 2nd Qtr Pts": winner.findAll('td')[1].text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update({"Loser 2nd Qtr Pts": loser.findAll('td')[1].text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update({"Winner 3rd Qtr Pts": winner.findAll('td')[2].text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update({"Loser 3rd Qtr Pts": loser.findAll('td')[2].text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update({"Winner 4th Qtr Pts": winner.findAll('td')[3].text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update({"Loser 4th Qtr Pts": loser.findAll('td')[3].text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update( {"Winner Total Pts": winner.findAll('td')[4].text.strip()})
        except IndexError:
            d[None] = None
        try:
            dictlist[url[i]].update( {"Loser Total Pts": loser.findAll('td')[4].text.strip()})
        except IndexError:
            d[None] = None
        for j in range(16):
            try:
                dictlist[url[i]].update( { table.findAll('th', {'scope': 'row'})[j].text.strip() + ' Away': table.findAll('td')[2*j].text.strip()})
            except IndexError:
                d[None] = None
            try:
                dictlist[url[i]].update({ table.findAll('th', {'scope': 'row'})[j].text.strip() + ' Home' : table.findAll('td')[2*j+1].text.strip()})
            except IndexError:
                d[None] = None
            try:
                dictlist[url[i]].update({table.findAll('th', {'scope': 'row'})[16].text.strip()+' Away': table.findAll('td')[32].text.strip()})
            except IndexError:
                d[None] = None
    z = {**dictlist, **d}
    # we then merge the two dictionaries so that it shows which values are missing
    
    return z


Using this function we can get the data from any array of game websites and it is generalized to prevent errors and obtain the right fields for the associated keys.

Note: Some games can go to overtime but I found that an easy fix on Excel. If you sort the total points by least to greatest we can see that the quarters won't add up to the total so I created another column for the winner and losers and summed the entries to get the actual total.

Collecting all the links would be time consuming as well so a function was made for that as well.
We can use BeautifulSoup for this too since the html for the game log page contains a tag called href which is the url for the webpages that we are interested in.

In [9]:
def get_links():
    """ All links have a certain pattern since they are all similar. However there is a part of the url that makes 
     it unique and so we obtain the links from the html """
    print("getting links...")
    teams = ['algoma', 'brock', 'carleton', 'guelph', 'lakehead', 'laurentian',
             'laurier', 'mcmaster', 'nipissing', 'ottawa', 'queens', 'ryerson',
             'toronto', 'waterloo', 'western', 'windsor', 'york']
    years = ['2014-15', '2015-16', '2016-17', '2017-18', '2018-19']
    original_url = 'http://oua.ca/sports/mbkb/'
    end_url = '?view=gamelog'
    href_list = []
    for year in years:
        for team in teams:
            current_url = original_url + year + '/teams/' + team + end_url
            r = requests.get(current_url)
            raw_html = r.content
            soup = BeautifulSoup(raw_html, 'html.parser')
            tables = soup.findAll('table')
            max_len = 0
            index = 0

            for i in range(len(tables)):
                tags = tables[i].findAll('a')
                if len(tags) > 0:
                    url = tags[0].get('href', None)
                    if "/boxscores/20" in url and len(tables[i]) > max_len:
                        index = i
                        max_len = len(tables[i])

            table = tables[index]

            tags = table.findAll('a')
            for tag in tags:
                url = re.sub("\.\.", original_url + year, tag.get('href', None))
                url += '?view=teamstats'
                href_list.append(url)

    print("done getting links")
    return href_list

Using both these functions together we can obtain our data and put it into a dataframe.

In [ ]:
    q = get_links()
    a = scrape(q)
    df = pd.DataFrame(a)
    df = df.T
    df = df.replace('\-', ' -- ', regex=True).astype(object) #This is used because when opening in Excel 
    #it automatically assumes that the FG is a date (e.g. FG: 10-12)
    df = df[['Away', 'FG Away', 'FG% Away', '3PT FG Away', '3PT FG% Away', 'FT Away', 'FT% Away', 'Rebounds Away',
             'Assists Away',
             'Turnovers Away', 'Points Off Turnovers Away', '2nd Chance Points Away', 'Points in the Paint Away',
             'Fastbreak Points Away', 'Bench Points Away',
             'Largest Lead Away', 'Time of Largest Lead Away', 'Home', 'FG Home',
             'FG% Home', '3PT FG Home', '3PT FG% Home', 'FT Home', 'FT% Home', 'Rebounds Home', 'Assists Home',
             'Turnovers Home',
             'Points Off Turnovers Home', '2nd Chance Points Home', 'Points in the Paint Home', 'Fastbreak Points Home',
             'Bench Points Home', 'Largest Lead Away', 'Time of Largest Lead Away', 'Trends Away', 'Winner', 'Winner 1st Qtr Pts',
             'Winner 2nd Qtr Pts', 'Winner 3rd Qtr Pts', 'Winner 4th Qtr Pts', 'Winner Total Pts', 'Loser', 'Loser 1st Qtr Pts', 'Loser 2nd Qtr Pts',
             'Loser 3rd Qtr Pts', 'Loser 4th Qtr Pts', 'Loser Total Pts']]

    df.to_csv('gamebygame2.csv', header=True)

<img src="images/excel.png"/>

Here is a sample of the output we got. There are 1171 rows and more than 50 columns.